In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

#  SparkSession 생성
spark = SparkSession \
    .builder \
    .appName("adam") \
    .getOrCreate()

# 외부 csv 데이터 불러온 후, 스키마 출력(inferSchema는 컬럼의 데이터 타입을 자동으로 추론)
data = spark.read.csv('./boston.csv', header=True, inferSchema=True)
data.printSchema()

# label인 medv를 제외한 컬럼을 가져오고 이를 하나의 피처 벡터 컬럼으로 변환 후 'features' 컬럼에 저장
feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

# 학습, 테스트셋 분리
train_data, test_data = data.randomSplit([0.8, 0.2], seed=20230921)

root
 |-- _c0: integer (nullable = true)
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [5]:
rf = RandomForestRegressor(featuresCol="features", labelCol="medv").setNumTrees(500)
print(rf.explainParams())

bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the featur

In [6]:
model = rf.fit(train_data)

In [7]:
# prediction 진행
predictions = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="medv", predictionCol='prediction')
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"rmse: {rmse:.4f}")
print(f"mae: {mae:.4f}")
print(f"r2: {r2:.4f}")

rmse: 3.7879
mae: 2.2942
r2: 0.8424
